In [8]:
import json
import pandas as pd
import numpy as np

import re
from sqlalchemy import create_engine

# from config import db_password
import time

In [9]:
# 1. Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    movie = dict(movie)
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
        if len(alt_titles) > 0:
            movie['alt_titles'] = alt_titles
        
        # merge column names
def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
            change_column_name('Adaptation by', 'Writer(s)')
            change_column_name('Country of origin', 'Country')
            change_column_name('Directed by', 'Director')
            change_column_name('Distributed by', 'Distributor')
            change_column_name('Edited by', 'Editor(s)')
            change_column_name('Length', 'Running time')
            change_column_name('Original release', 'Release date')
            change_column_name('Music by', 'Composer(s)')
            change_column_name('Produced by', 'Producer(s)')
            change_column_name('Producer', 'Producer(s)')
            change_column_name('Productioncompanies ', 'Production company(s)')
            change_column_name('Productioncompany ', 'Production company(s)')
            change_column_name('Released', 'Release Date')
            change_column_name('Release Date', 'Release date')
            change_column_name('Screen story by', 'Writer(s)')
            change_column_name('Screenplay by', 'Writer(s)')
            change_column_name('Story by', 'Writer(s)')
            change_column_name('Theme music composer', 'Composer(s)')
            change_column_name('Written by', 'Writer(s)')
    

        return movie

In [10]:
# 2 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

file_dir='C://Users/prana/OneDrive/Desktop/Movies-ETL/wikipedia-movies.json'

def extract_transform_load():
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    
    movies_metadata=pd.read_csv('C://Users/prana/OneDrive/Desktop/Movies-ETL/archive/movies_metadata.csv', low_memory=False)
    ratings=pd.read_csv('C://Users/prana/OneDrive/Desktop/Movies-ETL/archive/ratings.csv')

    # Open and read the Wikipedia data JSON file.
    with open(f'{file_dir}', mode='r') as file:
        wiki_movies_raw = json.load(file) 
    
    # 3. Write a list comprehension to filter out TV shows.
    wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie
                   and 'No. of episodes' not in movie]


    # 4. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    clean_movies=[clean_movie(movie) for movie in wiki_movies]

    # 5. Read in the cleaned movies list from Step 4 as a DataFrame.
    wiki_movies_df = pd.DataFrame(clean_movies)
    sorted(wiki_movies_df.columns.tolist())

    # 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try:
        wiki_movies_df['imdb_id']=wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        wiki_movies_df.dropduplicates(subset='imdb_id', inplace=True)
    except Exception as excp:
        print (excp)
        
    #  7. Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    wiki_keepcol=[column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum()<=len(wiki_movies_df)*0.9]
    wiki_movies_df=wiki_movies_df[wiki_keepcol]
    
    # 8. Create a variable that will hold the non-null values from the “Box office” column.
    bo_nonnull= wiki_movies_df['Box office'].dropna()
    
    # 9. Convert the box office data created in Step 8 to string values using the lambda and join functions.
    bo_nonnull=bo_nonnull.apply(lambda x:  ''.join(x) if type(x) == list else x)

    # 10. Write a regular expression to match the six elements of "form_one" of the box office data.
    form_one=r'\$\s*\d+\.?\d*\s*[mb]illi?on'
    
    # 11. Write a regular expression to match the three elements of "form_two" of the box office data.
    form_two=r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'
    bo_nonnull=bo_nonnull.str.replace(r'\$.*[---](?![a-z])', '$', regex=True)
    
    # 12. Add the parse_dollars function.
    def parse_dollar(s):
        #to return non string 
        if type(s)!=str:
            return np.nan
        
        #to return value in the form of $###.# million
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):
            
            #remove $sign and 'million'
            s=re.sub('\$|\s|[a-zA-Z]', '', s)
            
            #convert to float and multiply by million
            value=float(s)*10**6
            
            return value
            
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):
            
            #remove $sign and 'million'
            s=re.sub('\$|\s|[a-zA-Z]', '', s)
            
            #convert to float and multiply by billion
            value=float(s)*10**9
            
            return value
        
        elif re.match('r\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):
            s=re.sub('\$|,', '', s)
            
            value=float(s)
            return value
        
        else:
            return np.nan    
        
    # 13. Clean the box office column in the wiki_movies_df DataFrame.
    wiki_movies_df['bo_nonnull']=bo_nonnull.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    wiki_movies_df.drop('bo_nonnull', axis=1, inplace=True)
        
    # 14. Clean the budget column in the wiki_movies_df DataFrame.
    budget=wiki_movies_df['Budget'].dropna()
    budget=budget.map(lambda x: ' '.join(x) if type(x) == list else x)
    budget=budget.str.replace(r'\$.*[---](?![a-z]', '$', regex=True)
    budget=budget.str.replace(r'\[\d+\]\s*', '')
    
    
    wiki_movies_df['budget']=budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    wiki_movies_df.drop('Budget', axis=1, inplace=True)
    

    # 15. Clean the release date column in the wiki_movies_df DataFrame.
    release_date=wiki_movies_df['Release date'].dropna()
    release_date=release_date.map(lambda x: ' '.join(x) if type(x) == list else x)
    date_form_one=r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}'
    date_form_two=r'\d{4}.[01]\d.[123]\d'
    date_form_three=r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
    date_form_four=r'\d{4}'

    # 16. Clean the running time column in the wiki_movies_df DataFrame.
    running_time=wiki_movies_df['Running time'].dropna()
    running_time=running_time.map(lambda x: ' '.join(x) if type(x) == list else x)
    running_time_keep=running_time.str.extract(r'(\d+)\s*ho?u?r?s\s*(\d*)|(\d+)\s*m')
    wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
    wiki_movies_df.drop('Running time', axis=1, inplace=True)
    
    # Return three variables. The first is the wiki_movies_df DataFrame
    
    return wiki_movies_df, kaggle_metadata, ratings 

In [11]:
# 17. Create the path to your file directory and variables for the three files.
file_dir = 'C://Users/prana/OneDrive/Desktop/Movies-ETL/wikipedia-movies.json'
# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia_movies.json'
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [12]:
# 18. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = extract_transform_load()

'imdb_link'


KeyError: 'Box office'

In [ ]:
# 19. Set the wiki_movies_df equal to the wiki_file variable. 
wiki_movies_df = wiki_file

In [9]:
# 20. Check that the wiki_movies_df DataFrame looks like this. 
wiki_movies_df.head()

AttributeError: 'str' object has no attribute 'head'

In [10]:
# 21. Check that wiki_movies_df DataFrame columns are correct. 
wiki_movies_df.columns.to_list()

AttributeError: 'str' object has no attribute 'columns'